**Quick intro**

I wanted to see a notebook / tutorial that would take me through the **basics of working with time series**.  

Most notebooks I saw were either not very rigorous or they took me straight into price prediction using some methods, which is not what I needed right away.  

So I wrote my own introductory notebook. I acquired most of the information I used here through reading <a href='https://machinelearningmastery.com/introduction-to-time-series-forecasting-with-python/'>Introduction to Time Series Forecasting With Python</a> by Jason Brownlee (took a few days to read an implement).   

What this notebook is:  
- a good starting point for understanding time series data and how it differs from problems with other type of tabular data 
- a cookbook we can use for exploration when starting to work with a new dataset

What this notebook is not:
- it is not about prediction. It stops at exploration and understanding the data.
- it's not meant for advanced practitioners of asset price prediction - unless you want to revisit some concepts.  

### Contents

[1. Quick overview](#1.-Quick-overview)  
[2. Dataset description](#2.-Dataset-description)  
[3. Basic trading data visualization](#3.-Basic-trading-data-visualization)  
[4. Preprocessing](#4.-Preprocessing)  
[5. Feature engineering](#5.-Feature-engineering)  
[6. Typical time series visualizations](#6.-Typical-time-series-visualizations)  
[7. Power transforms](#7.-Power-transforms)  
[8. Temporal structure](#.8-Temporal-structure-of-time-series-data)

# 1. Quick overview

This dataset was made available on Kaggle as part of the <a href='https://www.kaggle.com/c/g-research-crypto-forecasting/overview'>G-Research Crypto forecasting competition</a>. The challenge proposed by G-Research was to predict price returns across a bundle of major cryptocurrencies for which we have approximately 3 years of historical price data.  

# 2. Dataset description

In [1]:
import pandas as pd
import numpy as np
import os 
import time

import matplotlib.pyplot as plt

from datetime import datetime

In [2]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

Available files (provided by the organizers of the competition):
- **train.csv**
- **asset_details.csv**
- example_sample_submission.csv
- example_test.csv
- supplemental_train.csv

Only the first two seem interesting for now.

In [3]:
start = time.time()

crypto_df = pd.read_csv(os.path.join(data_folder, 'train.csv'))

end = time.time()
print(end - start) # just ouf curiosity, see how long it takes to read such a large csv file

In [4]:
crypto_df.shape

There are approximately 24 million observations in our time series.

In [5]:
crypto_df.head(2)

**Data features**  

- **timestamp** - Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
- **Asset_ID** - uniquely identifies the traded coin
- **Count** - number of trades executed within the respective minute
- **Open, High, Low, Close** - the usual price details for a given unit of time. 
- **Volume** - amount of units of this coin traded in the particular minute
- **VWAP** - The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
- **Target** - Residual log-returns for the asset over a 15 minute horizon <- we know this from the competition's official description.

In [6]:
start = crypto_df.iloc[0].timestamp.astype('datetime64[s]')
end = crypto_df.iloc[-1].timestamp.astype('datetime64[s]')

print(f'Data from {start} until {end}')

We have approximately 3 years worth of data. This informs the type of time windows we can look at.  

For example, if we zoom out at an yearly resolution, we can only compare 2018, 2019 and 2020 (2021 is incomplete).

### Assets 

In the list of transactions, assets are referred to by Asset_ID. Let's look into **asset_details.csv** to see what these 'assets' are.

In [7]:
asset_details_df = pd.read_csv(os.path.join(data_folder, 'asset_details.csv'))

In [8]:
asset_details_df

An interesting selection of 14 crypto coins. I say interesting because, as a crypto investor myself, I wonder what IOTA and Dogecoin are doing in there.

# 3. Basic trading data visualization

Get a subset of our timeframe (the last 60 entries for example) and restrict the analysis to one asset (which makes it a univariate analysis). 

In [9]:
btc_mini_df = crypto_df[crypto_df.Asset_ID == 1].iloc[-60:]

In [10]:
btc_mini_df.head(4)

Let's transform the timestamp into the index of our data. This will facilitate some operations down the road.

In [11]:
btc_mini_df = btc_mini_df.set_index("timestamp")

We can use the candlestick function from the graph_objects package of plotly to visualize trading data as the common candlestick plot.

In [12]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_mini_df.index, 
                                     open=btc_mini_df['Open'], 
                                     high=btc_mini_df['High'], 
                                     low=btc_mini_df['Low'], 
                                     close=btc_mini_df['Close'])])
fig.show()

Candlestick plots look nice and fancy, but later we'll go beyond this into exploring trends, cycles and seasonality of the dataset.  

For now, it's just a way to get a quick look at our data, so we have a mental model of what we're dealing with.

# 4. Preprocessing  

Detecting features for which we have missing values is easy to do. We just look for null values per column.   

In [13]:
btc_df = crypto_df[crypto_df.Asset_ID == 1].set_index('timestamp')

btc_df.info(show_counts =True)

Or, more clearly:

In [14]:
btc_df.isna().sum()

Not a lot of data is missing. We can drop those rows, impute the values etc. I'm not going to use Target, so I won't go into this right now. 

The more covert missing data is when we don't have any information at all for a particular minute. Which means that **whole rows may be missing** from our dataset.  

We should have one row per minute per asset. Since we extracted the data for a single asset, we expect consecutive rows to have a difference of 60 seconds between their index values.  

First step is to look at the **time lag between consecutive entries** in our dataset.

In [15]:
# we start with 1 instead of 0 since there's nothing to compare the first entry to
(btc_df.index[1:]-btc_df.index[:-1]).value_counts().head()

We see that we have 78 instances where two consecutive entries are 120 seconds apart, instead of 60 sec, and so on. 

Because the gaps in data are so small, we can use a simple imputation method: fill in the missing data with the value from the most recent available minute.  

This is what the method = 'pad' parameter of the reindex function below does.    

In [16]:
btc_df = btc_df.reindex(range(btc_df.index[0],btc_df.index[-1]+60,60), method='pad')

Check the output

In [17]:
(btc_df.index[1:]-btc_df.index[:-1]).value_counts().head()

Great, no more time gaps in our dataset !  

We don't need the index as a timestamp anymore. For future analysis it will be easier to have it as a date.

In [18]:
btc_df['datetime'] = btc_df.apply(lambda r: np.float64(r.name).astype('datetime64[s]'), axis=1)

btc_df.set_index('datetime', inplace=True);

btc_df.head(3)

# 5. Feature engineering

According to the usual taxonomy of Machine Learning, we have supervised, unsupervised and reinforcement learning problems. I'm looking to apply supervised learning methods for this data.  

The data does not look like the typical supervised learning dataset. For each row, we expect to see several features and a label / value that we're trying to predict based on those features. 

Let's have a look again at our dataset below. Definitely not what we need, yet.

In [19]:
btc_df.head(3)

#### Rephrasing the dataset into a supervised learning dataset. 

In supervised learning datasets we have entries with a set of features (our **x**) and a label / value for our output variable which we want to predict (our **y**).  

In the case of time series, our output **y** is the price at time t. But what are the inputs for a possible prediction model ?   

To phrase it differently, for each entry in our dataset we need to have a set of feaures and a label (used for training and then outputted as a prediction of our model).  

This means we need to convert our original data:

| time | value |
| --- | --- |
| 1514764860 | 13850.176 | 
| 1514764920 | 13828.102 |
| 1514764980 | 13801.314 |
| ... | ... |

into something of this form:

| x | y |
| --- | --- |
| x1 | y1 |
| x2 | y2 |
| x3 | y3 |

#### Types of features (x) we can create from time series data:  
- **date time** features 
- **lag** features  
- **window** features   

### 5.1 Date time features

Again, our data looks like this:  

| timestamp | value |
| --- | --- |
| 1514764860 | 13850.176 | 
| 1514764920 | 13828.102 |
| 1514764980 | 13801.314 |
| ... | ... |

We can transform the timestamp into day, hour, min, this way creating three new features.  

What types of questions we can answer with this approach ? We can imagine a problem where we're trying to predict the price for a specific time on a specific day.  

This makes more sense if our data was temperatures throughout the day, where we know there are daily cycles, yearly cycles etc, rather than stocks, but we're just using this for illustration purposes for now.  

In [20]:
btc_mini_df = btc_df[-7201:].copy(deep=True)  # the 7201 comes from me cheating and looking ahead into the results, trying to get full days of data

In [21]:
btc_mini_df.head(3)

In [22]:
btc_mini_df['time'] = btc_mini_df.apply(lambda r:r.name, axis=1) # I need to move timestamp back into a column

# and parse it into year, month, day and hour
btc_mini_df['year'] = [btc_mini_df.iloc[i].time.year for i in range(len(btc_mini_df))]  
btc_mini_df['month'] = [btc_mini_df.iloc[i].time.month for i in range(len(btc_mini_df))]
btc_mini_df['day'] = [btc_mini_df.iloc[i].time.day for i in range(len(btc_mini_df))]
btc_mini_df['hour'] = [btc_mini_df.iloc[i].time.hour for i in range(len(btc_mini_df))]

In [23]:
btc_mini_df.head(3)

In [24]:
# average data per hour 
tmp = btc_mini_df.groupby(['year', 'month', 'day', 'hour']).mean()

# restore the multilevel index created by groupby into the year, month, day, hour columns that we created earlier
tmp.reset_index(inplace=True)

In [25]:
cols = ['year', 'month', 'day', 'hour', 'Close']

tmp[cols].head(5)

And that would be our training data for a supervised learning model.

Trying to predict coin prices based on day and hour will likely be a poor model.  

But we can enrich these features with additional ones. Here are a few candidates:
- Weekend or not.
- Season of the year.
- Business quarter of the year.  
etc  

I'm not convinced this would help much, so I'll explore further ways to build features.

### 5.2 Lag features  

This is the typical way in which time series data is tranformed into a supervised learning problem.  

The idea is to predict the value at the current timestamp based on the value from the previous timestamp(s).   

**Nomenclature:**  
- because we are predicting a single variable, this is called **univariate**  
- the number of previous time steps we use in our prediction is called the **width of the time window** or **the lag**
- if we predict only one future time step we are doing **one-step forecast**. We can also perform **multi-step forecast** and predict multiple next steps at once 
- this method is also called **sliding window**, with a window width of 1 in our case below.

Initial dataset:  

time | value  
\--------------   
t1&emsp;$\;$|  v1  
t2&emsp;$\;$|  v2  
t3&emsp;$\;$|  v3

becomes:    

x&emsp;| y  
\---------     
?$\;\;$|  v1      
v1$\;$|  v2  
v2$\;$|  v3  

The actual value for time is gone from our data. We don't care about it. We only care about the actual value at the previous point in time.

We use pandas' dataframe.shift() function, which shifts values vertically or horizontally, fills in with NaN values and leaves the index as it is.

In [26]:
tmp = btc_mini_df['Close'] # extract only the Close price

lag_df = pd.concat([tmp.shift(1, axis = 0), tmp], axis=1) # downward shift by 1 step 

# the original price series becomes the time t value, 
# while the downward shifted series is time t+1
lag_df.columns = ['Close(t)', 'Close(t+1)'] 

lag_df.head()

The same transformation as above, but this time we include the 3 previous values, for each future time t+1:

In [27]:
lag_df = pd.concat([tmp.shift(3), tmp.shift(2), tmp.shift(1), tmp], axis=1)

lag_df.columns = ['Close(t-2)', 'Close(t-1)', 'Close(t)', 'Close(t+1)'] # rename columns for easier read

lag_df.head()

Finally, the lag does not have to be linear: we don't necessarily have to use the previous k values. We can include in the lag window values from the same day last week or the same hour the day before etc. 

### 5.3 Window features  

We saw how we can add a window of a certain width as features to be used by our model to forecast the value at future time t+1.  

Besides using the raw values from the time window (i.e. Close(t-1), Close(t), etc), we can also compute *summary statistics* of these values and use them as features for prediction.  

The most common aggregate value is the mean of the lag window (also called **moving average** or **rolling mean**).

time | value  
\--------------   
t1&emsp;$\;$|  v1  
t2&emsp;$\;$|  v2  
t3&emsp;$\;$|  v3  
t4&emsp;$\;$|  v4

**shifted**  

time | value  
\--------------   
t1&emsp;$\;$|  ?  
t2&emsp;$\;$|  v1  
t3&emsp;$\;$|  v2  
t4&emsp;$\;$|  v3

shifted, applied **rolling mean** (aka moving average ) with a window size of 2  

time | value  
\--------------   
t1&emsp;$\;$|  ?  
t2&emsp;$\;$|  ?  
t3&emsp;$\;$|  (v1+v2) / 2  
t4&emsp;$\;$|  (v2+v3) / 2

Final dataset

| time | mean | Close(t+1) |  
| --- | --- | --- |   
t1&emsp;$\;$|  ?  |  v1  |  
t2&emsp;$\;$|  ?  |  v2  |  
t3&emsp;$\;$|  (v1+v2) / 2  |  v3  |  
t4&emsp;$\;$|  (v2+v3) / 2  |  v4  |  

The moving average can be used as a naive prediction model: predict for next day the average of the last *w* days (where *w* is the width of the moving average window). But first we are supposed to have *stationary* data (havign no obvious upward or downward trend and no seasonality), so we'll need some further preprocessing down the road. 

In [28]:
tmp = btc_mini_df['Close'] # extract only the Close price

lag_df = tmp.shift(1) # downward shift by 1

window = lag_df.rolling(window=2) # rolling window size of 2
means = window.mean() # compute the means for the rolling windows

new_df = pd.concat([means, tmp], axis=1) # concatenate the two series vertically

new_df.columns = ['mean(t-1,t)', 't+1'] # rename columns for easier reading

new_df.head()

There are of course many summary statistics we can use besides the average for the previous time window.    

Also, the window does not have to have a fixed width. It can also be a forever expanding window.  

| time | value |
| --- | --- | 
| t1 | 1 | 
| t2 | 2 | 
| t3 | 3 | 
| t4 | 4 | 
| t4 | 5 |

**expanding** rolling window  

| # |values |
| --- | --- |
| 0 | 1 2 |
| 1 | 1 2 3 |
| 2 | 1 2 3 4 |
| 4 | 1 2 3 4 5 |

final engineered dataset having summay statistics (min, mean and max window values) computed for the **expanding** rolling window

| # | min | mean | max | t+1 |
| --- | --- | --- | --- | --- |
| 0 | 1 | 1 | 1 | 2 |
| 1 | 1 | 1.5 | 2 | 3 |
| 2 | 1 | 2 | 3 | 4 |
| 3 | 1 | 2.5 | 4 | 5 |

In [29]:
window = tmp.expanding()

dataframe = pd.concat([window.min(), window.mean(), window.max(), tmp.shift(-1)], axis=1)

dataframe.columns = ['min', 'mean', 'max', 't+1']

print(dataframe.head(5))

In [30]:
# have a look at the initial dataset again, to verify that tha dataframe above is correct
tmp.head(3)

<a id='log_returns'></a>
#### Log returns feature engineering

A typical feature for assets price time series is the ratio between the current price and the price at the previous time point. This is usually computed as a log of the ratio in time series modelling, because that facilitates some apperations (additions, etc). In this case, they are called **log returns**.

To compute the log return, we can simply take the logarithm of the ratio between two consecutive prices. 

In [31]:
# helper function to compute the log returns
def log_returns(series, periods = 1):
    return np.log(series).diff(periods = periods)

In [32]:
f = plt.figure(figsize = (15,4))

lret_btc = log_returns(btc_mini_df.Close,1)[1:]

plt.plot(lret_btc)

plt.show()

To sum up, the three methods above are a few basic and common ways (by the book) in which time series are rephrased into a dataset on which we can apply supervised machine learning methods.  

That's it for feature engineering for now. I'll move on to visualizing the data.

# 6. Typical time series visualizations  

There are 3 types of information to explore in a time series through visualization:

<strong>temporal structure:</strong>  
&emsp;- line plots  
&emsp;- lag plots  
&emsp;- autocorrelation plots  
<strong>the distribution of observations:</strong>  
&emsp;- histograms  
&emsp;- density plots  
<strong>the change in distribution over time intervals:</strong>  
&emsp;- box and whisker plots  
&emsp;- heatmap plots

### 6.1 Line plots

In [33]:
btc_df.head(3)

In [34]:
#btc_df.plot(x='datetime', y='Close', figsize=(8,5))
btc_df.Close.plot(figsize=(20,5))
plt.title('Evolution of BTC price')
plt.show()

The plot is a bit dense, since it contains all the data we had (almost 3 years worth of data, on a 1-minuted resolution).  

But it's apparent that we don't see a noticeable pattern (no apperent pattern that repeats itself year after year, for example).  

For time series, it can be better to look at and compare plots from the same interval, like day-to-day, year-to-year etc.

Let's assume we're naive about crypto prices and we want to investigate whether there is an daily seasonality, so we want to plot it in a more informative way. 

In [35]:
print(btc_mini_df.iloc[0].time)
print(btc_mini_df.iloc[-2].time)
print(btc_mini_df.iloc[-1].time)

In [36]:
groups = btc_mini_df.groupby('day')

days = pd.DataFrame()

for name, group in groups:
    if name == 21: # skip the last day, which seems to be incomplete 
        continue
    days[name] = group.Close.values

days.plot(subplots=True, legend=False, figsize=(10,8), title='BTC price evolution throughout the day\n2021-09-16 to 2021-09-20');

If we have some domain knowledge, we can say that we're not surprised we don't see any apparent correlation between consecutive days.

### 6.2 Histograms and density plots  

Some linear time series forecasting methods assume a well-behaved distribution of observations (like a normal distribution). Before doing statistical tests to formally assess the assumption of normality, we can easily visualize our data as a histogram.

In [37]:
btc_df.Close.hist(figsize=(8,5))
plt.title('Histogram of closing prices for BTC')
plt.show()

We see the same information from the line plot and no further insight.

In [38]:
btc_df.Close.plot(kind='kde')
plt.title('Density plot of closing prices for BTC')
plt.show()

### 6.3 Box and whisker plots

We looked at the distribution of values across the whole timeframe (3 years worth of data).  

But it may be useful to examine the distribution in smaller time windows.  

Let's group our data by year and visualize it as box and whisker plots.

In [39]:
print(btc_df.iloc[0].name)
print(btc_df.iloc[-2].name)
print(btc_df.iloc[-1].name)

In [40]:
btc_df.groupby(pd.Grouper(freq='A')).size()

We don't have the same number of observations for each year. 2020 has one extra day, for example. 2018 is missing a single data point. And 2021 is quite incomplete. 
Just to proceed fast through this step, I'll use a quick dirty trick and join the data for each year while filling in missing values with NaN. It's not the best thing to do, but for the scope of this analysis it's good enough. 

In [41]:
groups = btc_df.groupby(pd.Grouper(freq='A')) # group by year

years = pd.DataFrame([])

for name, group in groups: # iterate through the years
    tmp = group.groupby(pd.Grouper(freq='D')).Close.mean() # compute the daily mean
    tmp.index = tmp.index.strftime('%m-%d') # transform the index into 'mm-dd' only
    
    years = years.join(tmp, rsuffix=name.year, how = "outer") # join together yearly series (on the 'mm-dd' index) 
    
years.boxplot(figsize=(8,6))

plt.title('Box and whiskers plots for BTC close prices\n years 2018 to 2020');

[Short recap]   
A box plot is interpretted like this:

- The middle 50% of the data is contained in the block itself. The upper edge (hinge) of the box indicates the 75th percentile of the data set, and the lower hinge indicates the 25th percentile. 
- The horizontal line inside the box indicates the median value of the data.
- If the median line within the box is not equidistant from the hinges, then the data is skewed.
- The small horizontal ends of the vertical lines (the "whiskers") indicate the minimum and maximum data values, unless outliers are present in which case the whiskers extend to a maximum of 1.5 times the inter-quartile range (the hight of the box).
- The points outside the ends of the whiskers are (suspected) outliers.  

Insights from the plot above:  
- median value for BTC changes slightly in the first 3 years
- 2018 and 2020 are quite rich in outliers compared to the other 2
- in 2021 BTC price spiked 

### 6.4 Heat map plots  

Another way to look at this data is to plot it as a 2D plot and color-encode price values. 

In [42]:
yrs = ['Close', 'Close2019', 'Close2020']

plt.matshow(years[yrs].dropna().T, interpolation=None, aspect='auto')

plt.title('BTC closing daily price\nyears 2018->2020')
plt.ylabel('years')
plt.xlabel('days')
plt.show()

The problem we see in the plot above is that matshow seems to scale the colors relative to the whole 3 years worth of data.  
We've seen in the box plots above that that 2021 has a much higher mean value than all previous years.  

So this upward going trend from one year to the next obscures the evolution of price throughout the year and makes it less visible.  

Therefore I'll first normalize (from min 0 to max 1) the closing price for each year and replot.

In [43]:
norm_years=(years-years.min())/(years.max()-years.min())

plt.matshow(norm_years[yrs].dropna().T, interpolation=None, aspect='auto')

plt.title('BTC closing daily price\nyears 2018->2020\normalized per year')
plt.ylabel('years')
plt.xlabel('days')

plt.show()

Now it's easier to spot maximum and minimum per year, but we still don't see an pattern across years.

How about a daily pattern ?  

Let's see a heatmap of price during the day for 10 consecutive days.

In [44]:
groups = btc_df.groupby(pd.Grouper(freq='D'))

days = pd.DataFrame([])

for i, (name, group) in enumerate(groups):
    tmp = group.Close
    tmp.index = tmp.index.strftime('%H:%M')
    
    days = days.join(tmp, rsuffix=name.year, how = "outer")
    
    if i == 9:
        print('break')
        break
    
plt.matshow(days.T, interpolation=None, aspect='auto')
plt.show()

Still not very informative. This doesn't seem to be the right way to look at assets prices and now we know.

### 6.5 Lag plots  

Time series data implies a relationship between the value at a time t+1 and values and previous points in time.  

The step size we take to go back in time is called **lag** (lag of 1, lag 2 etc).  

Pandas provides the lag plot method. Let's examine the plot first.

In [45]:
from pandas.plotting import lag_plot

lag_plot(btc_df.groupby(pd.Grouper(freq='D')).Close.mean())

plt.show()

We see a strong positive relation between Close price at t and Close price at t+1.  

Let's experiment with the lag value.

In [46]:
daily_df = btc_df.groupby(pd.Grouper(freq='D')).Close.mean()
daily_values = pd.DataFrame(daily_df.values)

lags = 8
columns = [daily_values]

for i in range(1,(lags + 1)):
    columns.append(daily_values.shift(i)) # downward shift by i positions

dataframe = pd.concat(columns, axis=1)

col_names = ['t'] + ['t-'+str(l) for l in range(1,(lags + 1))]

dataframe.columns = col_names

plt.figure(figsize=(16,6))

for i in range(1,(lags + 1)):
    ax = plt.subplot(240 + i)
    ax.set_title('t vs t-' + str(i))
    plt.scatter(x=dataframe['t'].values, y=dataframe['t-'+str(i)].values)

plt.show()

Observations: the price at t correlates quite strongly with the price at previous time points (from t-1 to t-8), for lower price values (approximately half of maximum price). Beyond this value, the correlation becomes weaker the more we go back in time.  

We know from the first line plot we looked at that Bitcoin price surged in 2021. Also, common knowledge of crypto assets market says that prices get very volatile when there is a lot of hype and more people get on the trading markets. So, when prices are high, we expect a lot of volatility (more abrupt and erratic price changes across days). This is what the plots above tell us too.  

So, what we conclude: tehre's a strong positive relationship with prices on previous days, but not so much when the market becomes hyped and volatile.

### 6.6 Autocorrelation plots

Lag plots showed a relationship between current price and price at previous time points.  

We can also quantify the relationship between the price and the actual lag value.  

For a lag=1, we can compute the correlation between current time step value and previous time step value. If we have n time steps in our data, we'll have n-1 correlation values. These values can be anywhere inthe interval [-1,1].  

-1 (strongest negative correlation)  
0 (no relationshop at all)  
1 (strongest positive correlation)  

This computation can be done for lag=1 to any lag value we want.

Pandas provides the autocorrelation_plot() function for this.  

Values outside the dotted lines are statistically significant.  

For many types of time series data this will look like a sinusoid, with a decreasing amplitude. It's not the case for asset prices (at least not for those with large liquidity  - traded by many people).  

In [47]:
from pandas.plotting import autocorrelation_plot

btc_days_df = btc_df.groupby(pd.Grouper(freq='D')).Close.mean()

autocorrelation_plot(btc_days_df)

plt.title('Autocorrelation plot\ndaily resolution')
plt.show()

We see a statistically significant correlation with up tp 200-ish previous days average prices. The more we go back in time, the lower the correlation, until it starts to become slightly negative (statistically significant).  

We know markets have a global evolution (bull / bear market) and local trends (short term excitement for the price of an asset and then slight corrections).

So, depending on the time resolution our data has, I expect the autocorrelation plot to capture the underlying long or short term trend. 

# 7. Power transforms 

Let's see the linear plot again.

In [48]:
#btc_df.plot(x='datetime', y='Close', figsize=(8,5))

month_sz = 60 * 24 * 30
year_sz = 12 * month_sz

btc_df.Close.plot(figsize=(20,5))

# tail-rolling average transform
rolling_m = btc_df.Close.rolling(window=month_sz)
rolling_m_mean = rolling_m.mean()
rolling_m_mean.plot(color='green')

rolling_y = btc_df.Close.rolling(window=year_sz)
rolling_y_mean = rolling_y.mean()
rolling_y_mean.plot(color='red')

plt.legend(['BTC price/min', '30-day rolling avg', '1-year rolling avg'])
plt.title('Evolution of BTC price')
plt.show()

<a id='non_stationary'></a>
We see that the data is non-stationary: there is an **increasing trend**, yet non-linear (the 1-year rolling average in the plot above, going up at least in the last two years) and a **seasonality** component (local ups and downs).  

Furthermore, data **variance** (difference between peaks and throughs in the local oscillaions) also changes (higher in the last year). 

These make it more difficult to model the data with classical statistical methods.

A common wasy to remove these and to improve the signal to noise ratio are power transforms.  

Two of the commonly used power transforms are: **square root** and **log transform**. 

### Square root

Square root helps bring the data into a linear trend and well-behaved distribution (i.e. Gaussian, uniform) *when* the data is quadratic.  

Here's an example on made up data.

In [49]:
quad_data = [i**2 for i in range(1,200)]
plt.figure(figsize=(15,6))

## quadratic data plots
# line plot
plt.subplot(221)
plt.plot(quad_data)
plt.title('Made up data (qudratic)')

# histogram
plt.subplot(223)
plt.hist(quad_data)

## square root data plots
# linear plots
sq_data = np.sqrt(quad_data)
plt.subplot(222)
plt.plot(sq_data)
plt.title('Data after transformation (squared root)')

# histogram
plt.subplot(224)
plt.hist(sq_data)

plt.show()

Here's the result on our real world data:

In [50]:
data = btc_df.Close
plt.figure(figsize=(15,6))

## quadratic data plots
# line plot
plt.subplot(221)
plt.plot(data)
plt.title('Real data (BTC closing price)')

# histogram
plt.subplot(223)
plt.hist(data)

## square root data plots
# linear plots
sq_data = np.sqrt(data)
plt.subplot(222)
plt.plot(sq_data)
plt.title('Data after transformation (squared root)')

# histogram
plt.subplot(224)
plt.hist(sq_data)

plt.show()

The squared root doesn't seem to help because our data is not quadratic.

### Log transform

Log transforms help when the data has an exponential trend, which is often likened to a hokey stick in asset price popular terminology.  

We could say we see something like a hockey stick starting in Jan 2021. Let's explore this.

In [51]:
data = btc_df.Close
plt.figure(figsize=(15,6))

## initial data plots
# line plot
plt.subplot(221)
plt.plot(data)
plt.title('Real data (BTC closing price)')

# histogram
plt.subplot(223)
plt.hist(data)

## square root data plots
# linear plots
sq_data = np.log(data)
plt.subplot(222)
plt.plot(sq_data)
plt.title('Data after transformation (log transform)')

# histogram
plt.subplot(224)
plt.hist(sq_data)

plt.show()

It doesn't look like the right power transformation for our data.

scipy package offers a method to automatically search for the most suitable power transformation for a dataset: <a href='https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html'>boxcox function</a>.

[Short overview]  
BoxCox procedure identifies the appropriate exponent (lambda = l) to use to transform data into a well-behaved distribution.    

The returned lambda value indicates the power to which all data should be raised.  

In order to do this, the Box-Cox power transformation searches from lambda = -5 to lambda = +5 until the best value is found.  

Best lambda means: the one for which the transformed data's standard deviation is the smallest. In this situation, the data has the highest likelihood – but not a guarantee – to be normally distributed.

Table 1: Common Box-Cox Transformations

| lambda | transformed Y |   
| --- | --- |   
| -2 | 	Y^(-2) = 1/Y^2
| -1 | 	Y^(-1) = 1/Y^1
| -0.5 | 	Y^(-0.5) = 1/(Sqrt(Y))
| | 	log(Y)
| 0.5 | 	Y^(0.5) = Sqrt(Y)
| 1 | 	Y^1 = Y
| 2 | 	Y^2

In [52]:
# automatically box-cox transform a time series
from scipy.stats import boxcox

plt.figure(figsize=(15,6))

## initial data plots
# line plot
plt.subplot(221)
plt.plot(data)
plt.title('Real data (BTC closing price)')

# histogram
plt.subplot(223)
plt.hist(data)

## transformed data plots
temp_df = pd.DataFrame(btc_df.Close)
temp_df.columns = ['Close']

temp_df['Close'], lambda_val = boxcox(temp_df['Close'])
print(f'best lambda: {lambda_val}')

# line plot
plt.subplot(222)
plt.plot(temp_df['Close'])

# histogram
plt.subplot(224)
plt.hist(temp_df['Close'])
plt.show()

The best power transformation for our data seems to be the reciprocal square root (lambda = -0.5), according to the boxcox method.  

The distribution we obtain is the closest to a normal distribution, so far. But the trend is not linear. We see what happens with data from previous bull markets (2018 highs become more pronounced after this transformation). Too bad we don't have data from December 2017, when the actual peak of the previous bull market happened for BTC prices. My guess is it would approach the level of July 2021 in a transformed plot.  

I won't explore this further for the moment, as I'm not convinced this is the right transformation we need for our data.

# 8. Temporal structure of time series data

## 8.1 White noise

In a [previous section](#log_returns) I computed the log returns for the BTC close price. This plot reminded me of white noise.  

We can formally investigate if it actually is white noise .  

A time series is white noise if the variables are independent and identically distributed with a mean of zero.  

White noise is important in time series forecasting for two reasons:  
1. **Prediction**: If a time series is white noise, then it's by definition random and cannot be predicted.
2. **Diagnosis**: The errors of a time series model should be white noise. What does this mean ? That the erros contain no information, as all the information from the time series was harnessed by the model itself. And the opposite ? If the erors are not white noise, the model can be improved further.

However, it's generally expected that any real like time series will contains a certain amount of white noise.

A series is *not* white noise if:
- the mean is non zero
- the variance changes over time
- the is a significant autocorrelation (with lagged values)

In [53]:
lret_btc = log_returns(btc_df.Close,1)[1:]

In [54]:
f = plt.figure(figsize = (15,4))

plt.plot(lret_btc)
plt.plot(lret_btc.index, [lret_btc.mean()] * len(lret_btc), color='red')
plt.legend(['BTC log returns', 'mean'])

plt.show()

In [55]:
print(lret_btc.describe())

Mean and std are almost 0. Since variance = std^2, it's also close to 0. 

Since we have a lot of data, we can split it into shorter time intervals and see if the summary stats change. 

In [56]:
groups = lret_btc.groupby(pd.Grouper(freq='A')) # group by year

years = pd.DataFrame([])

for name, group in groups: # iterate through the years
    tmp = group.groupby(pd.Grouper(freq='D')).mean() # compute the daily mean
    tmp.index = tmp.index.strftime('%m-%d') # transform the index into 'mm-dd' only
    
    years = years.join(tmp, rsuffix=name.year, how = "outer") # join together yearly series (on the 'mm-dd' index) 
    
years.boxplot(figsize=(8,6))
plt.ylim([-0.0002, 0.0002])
plt.title('Box and whiskers plots for log returns of BTC close prices\n years 2018 to 2021');

Let's see the autocorrelation for the log returns now.

In [57]:
lret_btc = log_returns(btc_df,1)[1:]

temp_df = lret_btc.groupby(pd.Grouper(freq='D')).Close.mean()

autocorrelation_plot(temp_df)

plt.title('Autocorrelation plot\nlog returns\ndaily resolution')
plt.ylim([-0.25, 0.25])
plt.show()

The spikes past the 95% (solid grey line) and 99% (dotted grey line) confidence levels look like a statistical fluke, in this context.

## 8.2 Random walk  

A time series is constructed through a *random walk* process as follows:  
y(t) = X(t-1) + rnd_step,  
where rnd_step is randomly selected from {-1, 1} and os is x(0).

In [58]:
from random import seed
from random import random

seed(101)

values = [-1 if random() < 0.5 else 1] # x(0)

for i in range(1, 1000):
    rnd_step = -1 if random() < 0.5 else 1
    y_t = values[i-1] + rnd_step
    values.append(y_t)

plt.figure(figsize=(8,7))

# linear plot
plt.subplot(211)    
plt.plot(values)
plt.title('Line plot of a generated random walk time series')

# correlogram
plt.subplot(212)    
autocorrelation_plot(values)
plt.title('Autocorrelation plot for a generated random walk time series')

plt.show()

In section [section 6.6](#6.6-Autocorrelation-plots) I plotted the autocorrelation for BTC close price. Let's see it again.

In [59]:
from pandas.plotting import autocorrelation_plot

btc_days_df = btc_df.groupby(pd.Grouper(freq='D')).Close.mean()

plt.figure(figsize=(8,4))
autocorrelation_plot(btc_days_df)

plt.title('Autocorrelation plot\ndaily resolution')
plt.show()

The autocorrelation plot of the closing price looks exactly like that of a random walk.  

In [section 7](#non_stationary) I mentioned our closing price time series looking non-stationary. It's time to apply formal methods to test this assumption.  

The *statsmodel* library provides the *adfuller()* method which implements the *Augmented Dickey-Fuller test*.  

In [60]:
# calculate the stationarity of our closing price data
from statsmodels.tsa.stattools import adfuller

# statistical test
result = adfuller(btc_days_df)

print(f'ADF result: {result[0]} p={result[1]:.3f}') 

print('Critical Values:')
for key, value in result[4].items():
    print(f'\t{key}: {value:.3f}')

Null hypothesis (H0) of the ADF test: time series is non-stationary.  
Based on the results, H0 cannot be rejected and the chances of this being a fluke are small.  

Then the next interesting question is how we can make it stationary.  

An easy method is to subtract the previous value for each time step t. That would be an obvious thing to do since we know our dataset looks like a random walk and we know that we obtained a random walk by using a linear function of Close(t-1) to predict Close(t).

In [61]:
diff_df = btc_days_df.diff()[1:]

plt.figure(figsize=(8,7))

# linear plot
plt.subplot(211)    
plt.plot(diff_df)
plt.title('Line plot of the diff time series')

# correlogram
plt.subplot(212)    
autocorrelation_plot(diff_df)
plt.title('Autocorrelation plot for the diff time series')
plt.ylim([-0.25, 0.25])
plt.show()

Sadly, it looks like we get white noise when we remove non-stationarity, which we know contains no structure that we can model and use for prediction.  

A naive model for white noise is:  
*Close(t) = Close(t-1)*  
This is also called the **persistence model**  
Just because we know the next value is a function of the current value but we have no way to build a model of this relationship.  

The persistence model will be the baseline for any future model.

In [62]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# prepare dataset
train_size = int(len(btc_days_df) * 0.5)
train, test = btc_days_df[0:train_size], btc_days_df[train_size:]

# persistence
preds = []
prev = train[-1]

for i in range(len(test)):
    preds.append(prev)
    prev = test[i]
    
rmse = sqrt(mean_squared_error(test, preds))
print(f'Persistence model RMSE: {rmse:.3f}')

In conclusion, I found out that this data is likely a random walk time series. I think that discovering this through analysis is more valuable than having read this commonly known fact directly from some article dealing with forecasting security prices over time. 

## 8.3 Time series decomposition  

A time series is conceptualized as having these types of components:
1. systematic components
    - level = overall average value 
    - trend = temporary upward or downward movement 
    - seasonality = a short-term cycle that repeats itself   
2. non-systematic components
    - random noise
    
The 4 components are thought to combine in two opssible ways into a time series:  
- additive  
    *Close(t) = level + trend + seasonality + noise*  
        
- multiplicative  
    *Close(t) = level * trend * seasonality * noise*  
    
In reality, time series data can have:
- both additive and multiplicative components  
- both upward and downward trends (especially security price data)  
- non-repeating and repeating cycles

In [63]:
from statsmodels.tsa.seasonal import seasonal_decompose

##whole data
data = btc_days_df
decomp = seasonal_decompose(data, model='multiplicative')

plt.figure(figsize=(20,12))

plt.subplot(421)
data.plot()
plt.ylabel('initial data')
plt.title('Whole dataset (3.5 years)')

plt.subplot(423)
decomp.trend.plot()
plt.ylabel('trend')

plt.subplot(425)
decomp.seasonal.plot()
plt.ylabel('seasonal')

plt.subplot(427)
decomp.resid.plot()
plt.ylabel('resdual')

##small window
data = btc_days_df[:160]
decomp = seasonal_decompose(data, model='multiplicative')

plt.subplot(422)
data.plot()
plt.ylabel('initial data')
plt.title('Smaller time window (0.5 years)')

plt.subplot(424)
decomp.trend.plot()
plt.ylabel('trend')

plt.subplot(426)
decomp.seasonal.plot()
plt.ylabel('seasonal')

plt.subplot(428)
decomp.resid.plot()
plt.ylabel('resdual')

plt.show()

You've reached the temporary end. This is work in progress. There is no conclusion yet. I continue to update this notebook as I read more.